<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Data%20Processing/Data%20Splitting%20and%20Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Dataset/Parasite.7z
!7z x Parasite.7z

--2021-05-14 05:04:09--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Dataset/Parasite.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20003284 (19M) [application/octet-stream]
Saving to: ‘Parasite.7z’

Parasite.7z         100%[===================>]  19.08M  69.5MB/s    in 0.3s    

2021-05-14 05:04:10 (69.5 MB/s) - ‘Parasite.7z’ saved [20003284/20003284]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 20003284 bytes (20 MiB)

Extracting archive: Parasite.7z
--
Path = Parasite.7z
Type = 7z
Physical Size = 20003284
He

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

import torchvision
from PIL import Image

In [ ]:
del data

In [3]:
data = pd.read_csv("/content/Dataset/all_meta_data_updated.csv")

In [4]:
data.species.replace({"Hymenolepis diminuta":"Hymenolepis spp",
                    "Hymenolepis nana":"Hymenolepis spp",
                    },inplace = True)
data['phylum'].replace({'Roundworms':'Nematoda','Flatworm':'Platyhelminthes'},inplace = True)
data['class'].replace({'\tConoidasida':'Conoidasida','Tapeworms':'Cestoda'},inplace = True)
data['species'].replace({'Entamoeba histolytica]':'Entamoeba histolytica'},inplace = True)
data['form'].replace({'rings':'trophozoites','decorticated egg':'egg','ookinete':'egg'},inplace = True)
data['sample'].replace({'intestinal tissue':'intestine', 'intestine ':'intestine','sputum':'lungs'},inplace = True)

In [5]:
data = [x for _,x in data.groupby(['phylum', 'class', 'genus', 'species', 'form', 'sample']) if len(x)>=8]

data = pd.concat(data,ignore_index=True).sample(frac = 1)

In [6]:
data = data[data['sample']!='na']
data = data[data['sample']!='liver']

In [7]:
labels = ['phylum', 'class', 'genus', 'species', 'form', 'sample']

In [8]:
train, valTest = train_test_split(
    data, test_size=0.25, random_state=0,stratify = data['species'])

In [9]:
for col in labels:
  print(15*'-'+col+15*'-')
  print(set(train[col].unique())-set(valTest[col].unique()))
  print(set(valTest[col].unique())-set(train[col].unique()))
  print(train[col].nunique(),valTest[col].nunique(),sep = ' -> ')
  print(valTest[col].nunique(),train[col].nunique(),sep = ' -> ')

---------------phylum---------------
set()
set()
5 -> 5
5 -> 5
---------------class---------------
set()
set()
7 -> 7
7 -> 7
---------------genus---------------
set()
set()
11 -> 11
11 -> 11
---------------species---------------
set()
set()
15 -> 15
15 -> 15
---------------form---------------
set()
set()
7 -> 7
7 -> 7
---------------sample---------------
set()
set()
4 -> 4
4 -> 4


In [ ]:
del test,val2

In [10]:
val, test = train_test_split(
    valTest, test_size=0.50, random_state=0,stratify = valTest['species'])

In [ ]:
val['form'].value_counts()

trophozoites    20
egg             18
oocysts         15
adult           12
gametocyte      10
larva            9
cyst             9
schizont         6
sarcocysts       1
Name: form, dtype: int64

In [11]:
for col in labels:
  print(15*'-'+col+15*'-')
  print(set(val[col].unique())-set(test[col].unique()))
  print(set(test[col].unique())-set(val[col].unique()))
  print(val[col].nunique(),test[col].nunique(),sep = ' -> ')
  print(test[col].nunique(),val[col].nunique(),sep = ' -> ')

---------------phylum---------------
set()
set()
5 -> 5
5 -> 5
---------------class---------------
set()
set()
7 -> 7
7 -> 7
---------------genus---------------
set()
set()
11 -> 11
11 -> 11
---------------species---------------
set()
set()
15 -> 15
15 -> 15
---------------form---------------
set()
set()
7 -> 7
7 -> 7
---------------sample---------------
set()
set()
4 -> 4
4 -> 4


In [12]:
for path in train['species'].unique():
  os.makedirs(f"/content/train/{path}")


for path in val['species'].unique():
  os.makedirs(f"/content/val/{path}")

for path in test['species'].unique():
  os.makedirs(f"/content/test/{path}")

In [ ]:
!rm -r train val test

In [13]:
t = []
t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomPerspective(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomVerticalFlip(p=1)]))


t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomVerticalFlip(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomPerspective(p=1)]))

In [14]:
def transform_save(data,index,folder):
  img = Image.open(data[8])
  images = [img]
  if folder not in ['val','test']:
    images.extend([T(img) for T in t])
  image_name = data[6].split('.')[0]
  for i, image in enumerate(images):
    image.convert('RGB').save(f"/content/{folder}/{data[3]}/{image_name}_{i}_{index}.jpg")
  values = [[data[0],data[1],data[2],data[3],data[4],data[5],f"./{folder}/{data[3]}/{image_name}_{i}_{index}.jpg"] for i in range(len(images))]
  return values


def get_dataframe(dataframe,folder = 'train'):
  new = []
  for i in dataframe.index:
    new.extend(transform_save(dataframe.loc[i,:].values,index =i,folder=folder))
  return pd.DataFrame(new,columns=['phylum','class','genus','species','form','sample','img_path'])

In [15]:
train = get_dataframe(train)

In [16]:
val = get_dataframe(val,folder = 'val')

In [17]:
test = get_dataframe(test,folder = 'test')

In [18]:
train.to_csv('/content/train/train.csv',index = False)
val.to_csv('/content/val/val.csv',index = False)
test.to_csv('/content/test/test.csv',index = False)

In [19]:
!7z a Train.7z train
!7z a Val.7z val
!7z a Test.7z test


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive:
  0M Scan           16 folders, 1141 files, 11227773 bytes (11 MiB)

Creating archive: Train.7z

Items to compress: 1157

  0%      9% 191 + train/Cyclospora cayetanensis/Cyclospora_SAF_Henry5_1_176.jpg                                                                         22% 316 + train/Entamoeba histolytica/Ehistdisp_troph_tric3_KS_1_18.jpg                                                                         33% 422 + train/Enterobius vermicularis/Evermic

In [1]:
import pandas as pd

In [2]:
labels = ['phylum', 'class', 'genus', 'species', 'form', 'sample']

train = pd.read_csv("/content/train/train.csv")
val = pd.read_csv("/content/val/val.csv")
test = pd.read_csv("/content/test/test.csv")

for col in labels:
  print(15*'-'+col+15*'-')
  print(train[col].nunique(),val[col].nunique(),test[col].nunique(),sep=' -> ')

---------------phylum---------------
5 -> 5 -> 5
---------------class---------------
7 -> 7 -> 7
---------------genus---------------
11 -> 11 -> 11
---------------species---------------
15 -> 15 -> 15
---------------form---------------
7 -> 7 -> 7
---------------sample---------------
4 -> 4 -> 4
